In [407]:
import pandas as pd
import numpy as np
import seaborn as sns
from biolearn.data_library import DataLibrary
from scipy.stats import pearsonr
import math

In [3]:
%%time 
wb_='data/cell/GSE41169_series_matrix.txt'
age=pd.read_table(wb_,skiprows=32+14,nrows=0,index_col=0).columns.str.strip('age: ').astype(float).tolist()
wb=pd.read_table(wb_,skiprows=32+14+26,index_col=0).drop('!series_matrix_table_end').round(3)

CPU times: user 7.6 s, sys: 936 ms, total: 8.53 s
Wall time: 8.61 s


In [4]:
%%time 
wb_t=wb.T
wb_t['age']=age
wb_cor=wb_t.corrwith(wb_t['age']).drop('age').sort_values()

CPU times: user 2min 2s, sys: 147 ms, total: 2min 2s
Wall time: 2min 2s


In [ ]:
hmr=pd.read_table('/labs/mpsnyder/moqri/epi_deconv/data/arc/hmr/neo',header=None,index_col=3,usecols=[0,1,2,3])
hmr['b']=hmr[0].str[3:].astype(int)*10**9+hmr[1]
hmr['e']=hmr[0].str[3:].astype(int)*10**9+hmr[2]
hmr=hmr.sort_values('b')

In [342]:
man=pd.read_table('../Blood-DNA-Methylation-Atlas/data/ref/HM450.hg38.manifest.tsv',index_col='Probe_ID')[['CpG_chrm','CpG_beg']]
man=man.dropna().copy()
man['CpG_beg']=man['CpG_beg'].astype(int)
man.columns=[0,1]
man['cg']=man.index
man=man.merge(wb_cor.rename('cor'),left_index=True,right_index=True)
man=man[man[0].isin('chr'+str(c) for c in range(1,23))].copy()
man['pos']=man[0].str[3:].astype(int)*10**9+man[1]
man=man.sort_values('pos')

In [422]:
cg=wb_cor[wb_cor>.17].head(1).index[0]
pearsonr(wb_t[cg],wb_t['age'])

PearsonRResult(statistic=0.17000257154587675, pvalue=0.09954625549702951)

In [424]:
n=len(wb_t)
r=.17
(n-2)**(1/2)*r/(1-r**2)**(1/2)

1.66363641734963

In [394]:
man=man[man['cor']>.170].copy()

In [395]:
%%time
l=[]
cur=0
for i in range(len(man)):
    if cur>=len(hmr):
        l.append(-1)
    else: 
        pos=man['pos'].iloc[i]
        cg=man['cg'].iloc[i]
        b=hmr['b'].iloc[cur]
        e=hmr['e'].iloc[cur]        
        if pos<b:
            l.append(-1)
        if pos>=b and pos<=e:
            l.append(cur)
        if pos>e:
            while (pos>hmr['e'].iloc[cur]):
                cur+=1
                b=hmr['b'].iloc[cur]
                e=hmr['e'].iloc[cur] 
            if pos>=b and pos<=e:
                l.append(cur)
            else:
                l.append(-1)

CPU times: user 3.4 s, sys: 32 ms, total: 3.43 s
Wall time: 3.42 s


In [396]:
man['l']=l
mg=pd.DataFrame(man.groupby('l')[0].count())
mg=mg[mg[0]>=4].copy()
print(len(mg))
mg['cor']=man.groupby('l')['cor'].mean()
hmr.iloc[mg.sort_values('cor').tail(10).index][[0,1,2]]

2628


,0,1,2
3,,,
HYPO15803,chr16,693607,696132
HYPO1989,chr1,52842501,52844181
HYPO24516,chr19,52492573,52493670
HYPO41672,chr7,130733013,130735171
HYPO2189,chr1,68046821,68047168
HYPO25032,chr2,11669594,11670306
HYPO37891,chr6,11042674,11044965
HYPO26321,chr2,105398173,105399417
HYPO39239,chr6,127514450,127516914


In [397]:
%%time 
man=man[man['l']>0].copy()
man['bin']=man['cor']>0
cont=[]
c=0
last=False
last_l=-1
for i,b in man[['l','bin']].iterrows():
    if (b['bin']):
        if (b['l']>last_l or last==False):
            c+=1
            last_l=b['l']
        cont.append(c)
        last=True  
    else:
        cont.append(-1)
        last=False

CPU times: user 2.26 s, sys: 25 ms, total: 2.29 s
Wall time: 2.29 s


In [400]:
man['cont']=cont
mgc=pd.DataFrame(man.groupby('cont')[0].count())
mgc['chr']=man.groupby('cont')[0].first()
mgc['b']=man.groupby('cont')[1].first()
mgc['e']=man.groupby('cont')[1].last()
mgc['cor']=man.groupby('cont')['cor'].mean()
topl=mgc[mgc[0]>=4].sort_values('cor')
topl.tail(10)

,0,chr,b,e,cor
cont,,,,,
8504,6,chr16,693924,694938,0.532847
507,6,chr1,52842924,52843588,0.533822
10573,4,chr19,52492829,52493363,0.552881
4810,10,chr7,130733487,130734723,0.577275
564,5,chr1,68046855,68047161,0.581576
1259,4,chr2,11669870,11670276,0.586686
3752,5,chr6,11043397,11044740,0.613848
1565,5,chr2,105398492,105399359,0.614469
4171,5,chr6,127514604,127515013,0.624143


In [ ]:
%%time
mono_='data/cell/GSE56046_methylome_normalized.txt'
mono=pd.read_table(mono_,index_col=0,usecols=[0]+list(range(1,201,2)))
p2=pow(2,mono).round(3)
mono=p2/(p2+1)
age=pd.read_table('https://ftp.ncbi.nlm.nih.gov/geo/series/GSE56nnn/GSE56046/matrix/GSE56046_series_matrix.txt.gz',skiprows=46,
             index_col=0).iloc[10].str.strip('age (yrs): ').astype(int)
mono_t=mono.T
mono_t['age']=age.head(100).values
mono_cor=mono_t.corrwith(mono_t['age']).drop('age').sort_values()

In [ ]:
%%time
t_='data/cell/GSE56581_methylome_normalized.txt'
t=pd.read_table(t_,index_col=0,usecols=[0]+list(range(1,201,2)))
p2=pow(2,t).round(3)
t=p2/(p2+1)
age=pd.read_table('https://ftp.ncbi.nlm.nih.gov/geo/series/GSE56nnn/GSE56581/matrix/GSE56581_series_matrix.txt.gz',skiprows=40,
             index_col=0).iloc[10].str.strip('age (yrs): ').astype(int)
t_t=t.T
t_t['age']=age.head(100).values
t_cor=t_t.corrwith(t_t['age']).drop('age').sort_values()

In [ ]:
%%time
liver_='data/cell/GSE48325_series_matrix.txt'
liver=pd.read_table(liver_,skiprows=27+33,index_col=0).round(3)
age=pd.read_table(liver_,skiprows=27,nrows=50,index_col=0).iloc[8].str.strip('age: ').astype(int)
liver_t=liver.T
liver_t['age']=age.values
liver_cor=liver_t.corrwith(liver_t['age']).drop('age').sort_values()

In [ ]:
dd=pd.concat([wb_cor,t_cor,mono_cor,liver_cor],axis=1).drop('!series_matrix_table_end')
dd.corr()

In [ ]:
t_hm=pd.read_table('../Blood-DNA-Methylation-Atlas/data/merge/hm450/cd4.meth',header=None,skiprows=860,index_col=3)[[0,1,4]]
m_hm=pd.read_table('../Blood-DNA-Methylation-Atlas/data/merge/hm450/mono.meth',header=None,skiprows=860,index_col=3)[[0,1,4]]